# Quick Start Guide - Emotion-Controlled Text Generation

This notebook provides a quick demonstration of the emotion-controlled text generation system.

**Prerequisites:** All models must be trained first using notebooks 01 and 02.

In [ ]:
import sys
sys.path.append('..')

from utils.emotion_predictor import EmotionPredictor
from utils.text_generation import EmotionControlledGenerator
from utils.emotion_mapping import TARGET_EMOTIONS
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful!")

## 1. Emotion Classification Demo

In [ ]:
# Load emotion classifier
classifier = EmotionPredictor('../models/emotion_classifier_roberta')

# Test samples
test_texts = [
    "I'm so excited about the weekend!",
    "This is really frustrating and annoying.",
    "I feel terrible about what happened.",
    "That movie was absolutely terrifying!",
    "Wow, I didn't see that coming!"
]

print("Emotion Classification Results:\n")
for text in test_texts:
    result = classifier.predict_emotion(text, return_all_scores=True)
    print(f"Text: {text}")
    print(f"Predicted: {result['emotion'].upper()} (confidence: {result['confidence']:.2f})")
    print()

## 2. Emotion-Controlled Generation Demo

In [ ]:
# Load generator (using token-based model)
generator = EmotionControlledGenerator(
    model_path='../models/gpt2_tokens',
    conditioning_method='token'
)

# Test context
context = "How was your day?"

print(f"Context: '{context}'\n")
print("="*70)

# Generate with all emotions
for emotion in TARGET_EMOTIONS:
    response = generator.generate_response(
        context=context,
        target_emotion=emotion,
        max_length=80,
        temperature=0.8
    )[0]
    
    # Verify emotion
    predicted_emotion = classifier.predict_emotion(response)
    match = "✓" if predicted_emotion == emotion else "✗"
    
    print(f"\n[{emotion.upper()}] {match}")
    print(f"Generated: {response}")
    print(f"Detected: {predicted_emotion}")

## 3. Interactive Generation

In [ ]:
def generate_emotional_response(context, emotion, model='token'):
    """
    Generate and validate emotion-controlled response.
    """
    model_paths = {
        'baseline': '../models/gpt2_baseline',
        'prefix': '../models/gpt2_prefix',
        'token': '../models/gpt2_tokens',
        'lora': '../models/gpt2_lora'
    }
    
    gen = EmotionControlledGenerator(
        model_path=model_paths[model],
        conditioning_method=model
    )
    
    response = gen.generate_response(
        context=context,
        target_emotion=emotion,
        temperature=0.7,
        max_length=100
    )[0]
    
    predicted = classifier.predict_emotion(response)
    
    return response, predicted

# Try it out!
my_context = "I heard you got some news today."
my_emotion = "happy"

response, detected = generate_emotional_response(my_context, my_emotion)

print(f"Context: {my_context}")
print(f"Target: {my_emotion}")
print(f"\nGenerated: {response}")
print(f"\nDetected emotion: {detected}")
print(f"Match: {'✓ Success!' if detected == my_emotion else '✗ Mismatch'}")

## 4. Compare All Methods

In [ ]:
context = "What do you think about the situation?"
emotion = "angry"

print(f"Context: '{context}'")
print(f"Target Emotion: {emotion.upper()}\n")
print("="*70)

for method in ['baseline', 'prefix', 'token', 'lora']:
    try:
        response, detected = generate_emotional_response(context, emotion, method)
        match = "✓" if detected == emotion else "✗"
        print(f"\n[{method.upper()}] {match}")
        print(f"Response: {response}")
        print(f"Detected: {detected}")
    except Exception as e:
        print(f"\n[{method.upper()}] Error: {e}")

## 5. Batch Generation

In [ ]:
# Generate multiple responses for different contexts
test_cases = [
    ("How's the project going?", "happy"),
    ("Did you hear the announcement?", "surprise"),
    ("I need to talk to you about something.", "fear"),
    ("What happened at the meeting?", "angry"),
]

generator = EmotionControlledGenerator(
    '../models/gpt2_tokens',
    conditioning_method='token'
)

print("Batch Generation Results\n")
print("="*70)

for i, (ctx, emo) in enumerate(test_cases, 1):
    response = generator.generate_response(
        context=ctx,
        target_emotion=emo,
        temperature=0.7
    )[0]
    
    detected = classifier.predict_emotion(response)
    match = "✓" if detected == emo else "✗"
    
    print(f"\n{i}. {match} Target: {emo.upper()}")
    print(f"   Context: {ctx}")
    print(f"   Response: {response}")
    print(f"   Detected: {detected}")

## Summary

This notebook demonstrated:

✅ Emotion classification using fine-tuned RoBERTa

✅ Emotion-controlled generation with GPT-2

✅ Comparison of different conditioning methods

✅ Validation of generated emotions

**Next Steps:**
- Run notebooks 01-03 for full training and evaluation
- Experiment with different contexts and emotions
- Analyze results in `results/evaluation_results.csv`